In [1]:
import bayesfast as bf
import numpy as np

In [2]:
D = 48 # number of dims
a = 5.
lower = np.full(D, -100.) # lower bound of the prior
upper = np.full(D, 100.) # upper bound of the prior
bound = np.array((lower, upper)).T
diff = bound[:, 1] - bound[:, 0]
const = np.sum(np.log(diff)) # normalization of the flat prior

def logp(x):
    _a = 1 / ((x + a)**2 + 1)
    _b = 1 / ((x - a)**2 + 1)
    return (np.sum(np.log(_a + _b), axis=-1) + x.shape[-1] * np.log(0.5 / np.pi)
            - const)

def grad(x):
    _a = 1 / ((x + a)**2 + 1)
    _b = 1 / ((x - a)**2 + 1)
    return -2 * ((x + a) / ((x + a)**2 + 1)**2 + 
                 (x - a) / ((x - a)**2 + 1)**2) / (_a + _b)

In [3]:
from distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=8, threads_per_worker=1)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:33107 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 126.70 GB


In [4]:
den = bf.DensityLite(logp=logp, grad=grad, input_size=D, input_scales=bound,
                     hard_bounds=True)
sample_trace = {'n_chain': 8, 'n_iter': 2500, 'n_warmup': 1000}
rec = bf.Recipe(density=den, sample={'sample_trace': sample_trace,
                'random_state': 0}, post={}, client=client)

In [5]:
rec.run()

 CHAIN #0 : sampling proceeding [ 500 / 2500 ], last 500 samples used 19.37 seconds. (warmup)
 CHAIN #1 : sampling proceeding [ 500 / 2500 ], last 500 samples used 25.92 seconds. (warmup)
 CHAIN #3 : sampling proceeding [ 500 / 2500 ], last 500 samples used 29.36 seconds. (warmup)
 CHAIN #2 : sampling proceeding [ 500 / 2500 ], last 500 samples used 33.22 seconds. (warmup)
 CHAIN #4 : sampling proceeding [ 500 / 2500 ], last 500 samples used 35.18 seconds. (warmup)
 CHAIN #6 : sampling proceeding [ 500 / 2500 ], last 500 samples used 37.89 seconds. (warmup)
 CHAIN #7 : sampling proceeding [ 500 / 2500 ], last 500 samples used 40.87 seconds. (warmup)
 CHAIN #0 : sampling proceeding [ 1000 / 2500 ], last 500 samples used 22.14 seconds. (warmup)
 CHAIN #3 : sampling proceeding [ 1000 / 2500 ], last 500 samples used 17.33 seconds. (warmup)
 CHAIN #5 : sampling proceeding [ 500 / 2500 ], last 500 samples used 51.15 seconds. (warmup)
 CHAIN #1 : sampling proceeding [ 1000 / 2500 ], last 500 

/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/glo

 ***** PostStep finished. ***** 



In [6]:
rec.get().logz, rec.get().logz_err # fiducial value: logz = -254.627

(-254.66024884466287, 0.08951702789853067)